In [25]:
import gurobipy as gp
import pandas as pd
from gurobipy import GRB

routes = pd.read_csv('https://raw.githubusercontent.com/decision-spot/technician_assignment/main/routes.csv')
routes

order_routes = pd.read_csv('https://raw.githubusercontent.com/decision-spot/technician_assignment/main/orders.csv')
order_routes


earliest_time = 0  # this is the reference for start time. Assume 7:00 AM is minute 0
latest_time = 600  # 10 hours later, 5:00 PM is minute 600
resource_limit = 3  # number of available resources


order_route_dict = dict(zip(order_routes['Customer Name'], order_routes['Route ID']))  # a(o,r)
routes_dict = routes.set_index('Route ID').to_dict(orient='index')  # this will help us in creating the constraints
r_set = routes_dict.keys()  # R
v_set = set(range(resource_limit))  # V
rv_set = {(r, v) for r in r_set for v in range(resource_limit)}  # pair of (r,v) index

In [26]:
mdl = gp.Model('resource_assignment')

# Variables
u = mdl.addVars(order_route_dict.keys(), vtype=GRB.BINARY, name='u')  # tracks whether order o is covered
x = mdl.addVars(r_set, vtype=GRB.BINARY, name='x')  # tracks whether route r is used
y = mdl.addVars(rv_set, vtype=GRB.BINARY, name='y')  # tracks whether route r is assigned to van v
z = mdl.addVars(v_set, vtype=GRB.BINARY, name='z')  # tracks whether van v is used
s = mdl.addVars(r_set, lb=earliest_time, ub=latest_time, vtype=GRB.CONTINUOUS, name='s')  # tracks start time of route r
e = mdl.addVars(r_set, lb=earliest_time, ub=latest_time, vtype=GRB.CONTINUOUS, name='e')  # tracks end time of route r

In [27]:
for o,r in order_route_dict.items():
    mdl.addConstr(x[r]==u[o], f'order_cov_{o}')

mdl.addConstrs((y.sum(r,'*') == x[r] for r in routes_dict), 'route_cov')
MAX_ROUTES = len(routes)
mdl.addConstrs((y.sum('*', v) <= z[v]*MAX_ROUTES  for v in v_set), 'lb_y_z')
mdl.addConstrs((z[v] <= y.sum('*', v) for v in v_set), 'ub_y_z')
mdl.addConstrs((s[r] >= routes_dict[r]['Earliest Start Time'] for r in r_set), 'lb_s')
mdl.addConstrs((s[r] <= routes_dict[r]['Latest Start Time'] for r in r_set),  'ub_s')
mdl.addConstrs((e[r] >= routes_dict[r]['Earliest End Time'] for r in r_set), 'lb_e')
mdl.addConstrs((e[r] <= routes_dict[r]['Latest End Time'] for r in r_set), 'ub_e')
mdl.addConstrs((e[r] == s[r]+routes_dict[r]['Total Time'] for r in r_set), 'rel_s_e')

rh_set = set((r1, r2) for r1 in r_set for r2 in r_set if r1 != r2)  # pair of (r,h) index
delta = mdl.addVars(rh_set, vtype=GRB.BINARY, name='d')
# Relation between any pair of routes assigned to a van
big_m = latest_time
for (r, h) in rh_set:
    if r < h:
        mdl.addConstr(s[h] >= e[r] - big_m * (1 - delta[r, h]), f'rbh_{r}_{h}')  # r before h
        mdl.addConstr(s[r] >= e[h] - big_m * (1 - delta[h, r]), f'hbr_{r}_{h}')  # h before r
        
        for v in v_set:
            mdl.addConstr(y[r, v] + y[h, v] - 1 <= delta[r, h] + delta[h, r], f'rel_d_y_{r}_{h}_{v}')
        

In [28]:
mdl.ModelSense = GRB.MINIMIZE
# weight=1: minimize, -1: maximize
mdl.setObjectiveN(u.sum(), index=0, priority=2, name='order_coverage', weight=-1)
mdl.setObjectiveN(z.sum(), index=1, priority=1, name='resource_usage', weight=1)

In [29]:
mdl.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 93 rows, 60 columns and 280 nonzeros
Model fingerprint: 0x16982760
Variable types: 10 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 6e+02]
  RHS range        [1e+00, 6e+02]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve...
---------------------------------------------------------------------------

Presolve removed 64 rows and 29 columns
Presolve time: 0.00s
Presolved: 29 rows, 31 columns, 110 no

In [30]:
res_df = pd.DataFrame()
status = mdl.status
if status == GRB.OPTIMAL:
    res_list = []
    for k, val in y.items():
        if val.x > 0.5:
            rid, n = k  # route id, resource number
            r = routes_dict[rid]
            st = s[rid].x  # start time
            rows = {'Resource Number': n, 'Route Number': rid, 'Start Time': st}
            rows.update(r)
            res_list.append(rows)
    res_df = pd.DataFrame(res_list)
    res_df = res_df.sort_values(by=['Resource Number', 'Start Time', 'Route Number'])
    uncovered_orders = {o for o in order_route_dict if u[o].x < 0.5}
    print(f'Uncovered orders: {uncovered_orders}') if uncovered_orders else print('All Covered!')
elif status in (GRB.INF_OR_UNBD, GRB.INFEASIBLE):
    print('Model is either infeasible or unbounded.')
else:
    print(f'Status is {status} which is unexpected.')
res_df

All Covered!


,Resource Number,Route Number,Start Time,Technician Name,Origin Location,Total Travel Time,Total Processing Time,Total Time,Earliest Start Time,Latest Start Time,Earliest End Time,Latest End Time,Num Jobs
1,0,4,0.0,Ed,Heidelberg,134.0,60.0,194.0,0.0,113.0,194.0,307.0,1
4,0,5,315.0,Flor,Freiburg im Breisgau,90.0,60.0,150.0,195.0,315.0,345.0,465.0,1
2,1,2,0.0,Bob,Heidelberg,100.0,30.0,130.0,0.0,100.0,130.0,230.0,1
3,1,3,178.0,Doris,Freiburg im Breisgau,141.0,120.0,341.0,58.0,178.0,399.0,519.0,2
0,2,1,0.0,Albert,Heidelberg,334.0,90.0,570.0,0.0,6.0,570.0,600.0,2


In [31]:
def resource_assignment(routes, order_routes, resource_limit):
    order_route_dict = dict(zip(order_routes['Customer Name'], order_routes['Route ID']))  # a(o,r)
    routes_dict = routes.set_index('Route ID').to_dict(orient='index')  # this will help us in creating the constraints
    r_set = routes_dict.keys()  # R
    v_set = set(range(resource_limit))  # V
    rv_set = {(r, v) for r in r_set for v in range(resource_limit)}  # pair of (r,v) index
    rh_set = set((r1, r2) for r1 in r_set for r2 in r_set if r1 != r2)  # pair of (r,h) index

    mdl = gp.Model('resource_assignment')

    # Variables
    u = mdl.addVars(order_route_dict.keys(), vtype=GRB.BINARY, name='u')
    x = mdl.addVars(routes_dict.keys(), vtype=GRB.BINARY, name='x')
    y = mdl.addVars(rv_set, vtype=GRB.BINARY, name='y')
    z = mdl.addVars(v_set, vtype=GRB.BINARY, name='z')
    s = mdl.addVars(r_set, lb=earliest_time, ub=latest_time, vtype=GRB.CONTINUOUS, name='s')
    e = mdl.addVars(r_set, lb=earliest_time, ub=latest_time, vtype=GRB.CONTINUOUS, name='e')
    delta = mdl.addVars(rh_set, vtype=GRB.BINARY, name='d')
    
    # Constraints
    # Relation between x and u
    for o, r in order_route_dict.items():
        mdl.addConstr(x[r] == u[o], f'order_cov_{o}')
        
    # Relation between y and x
    mdl.addConstrs((y.sum(r, '*') == x[r] for r in routes_dict), 'route_cov')
    
    # LB relation between y and z
    mdl.addConstrs((y[r, v] <= z[v] for (r, v) in rv_set), 'lb_y_z')

    # UB relation between y and z
    mdl.addConstrs((y.sum('*', v) >= z[v] for v in v_set), 'ub_y_z')
    
    # LB of s
    mdl.addConstrs((s[r] >= routes_dict[r]['Earliest Start Time'] for r in r_set), 'lb_s')

    # UB of s
    mdl.addConstrs((s[r] <= routes_dict[r]['Latest Start Time']  for r in r_set), 'ub_s')

    # LB of e
    mdl.addConstrs((e[r] >= routes_dict[r]['Earliest End Time'] for r in r_set), 'lb_e')

    # UB of e
    mdl.addConstrs((e[r] <= routes_dict[r]['Latest End Time'] for r in r_set), 'ub_e')

    # relation between s and e
    mdl.addConstrs((s[r] + routes_dict[r]['Total Time'] == e[r]  for r in r_set), 'rel_s_e')
    
    # Relation between any pair of routes assigned to a van
    for (r, h) in rh_set:
        if r < h:
            mdl.addConstr((delta[r,h] == 1) >> (s[h] >= e[r]), f'rbh_{r}_{h}')  # r before h
            mdl.addConstr((delta[h,r] == 1) >> (s[r] >= e[h]), f'hbr_{r}_{h}')  # h before r
            for v in v_set:
                mdl.addConstr(y[r, v] + y[h, v] - 1 <= delta[r, h] + delta[h, r], f'rel_d_y_{r}_{h}_{v}')

    # Objectives
    mdl.ModelSense = GRB.MINIMIZE
    # weight=1: minimize, -1: maximize
    mdl.setObjectiveN(u.sum(), index=0, priority=2, name='order_coverage', weight=-1)
    mdl.setObjectiveN(z.sum(), index=1, priority=1, name='resource_usage', weight=1)
    mdl.setParam(GRB.Param.OutputFlag, 0)  # disable printing the logs
    mdl.optimize()

    # create output
    res_df = pd.DataFrame()
    status = mdl.status
    if status == GRB.OPTIMAL:
        res_list = []
        for k, val in y.items():
            if val.x > 0.5:
                rid, n = k  # route id, resource number
                r = routes_dict[rid]
                st = s[rid].x  # start time
                rows = {'Resource Number': n, 'Route Number': rid, 'Start Time': st}
                rows.update(r)
                res_list.append(rows)
        res_df = pd.DataFrame(res_list)
        res_df = res_df.sort_values(by=['Resource Number', 'Start Time', 'Route Number'])
        uncovered_orders = {o for o in order_route_dict if u[o].x < 0.5}
        print(f'Uncovered orders: {uncovered_orders}') if uncovered_orders else print('All Covered!')
    elif status in (GRB.INF_OR_UNBD, GRB.INFEASIBLE):
        print('Model is either infeasible or unbounded.')
    else:
        print(f'Status is {status} which is unexpected.')
    return res_df

In [32]:
# run the model with different resource limits
all_outputs = []
for limit in range(1,5):
    print(f'**************\nRunning the model with {limit} vans\n**************')
    res = resource_assignment(routes, order_routes, limit)
    res['Resource Limit'] = limit
    all_outputs.append(res)
output_df = pd.concat(all_outputs)
output_df = output_df[["Resource Limit"] + [col for col in output_df.columns if col != "Resource Limit"]]

**************
Running the model with 1 vans
**************
Uncovered orders: {'Customer3', 'Customer1', 'Customer5', 'Customer7'}
**************
Running the model with 2 vans
**************
Uncovered orders: {'Customer1', 'Customer7'}
**************
Running the model with 3 vans
**************
All Covered!
**************
Running the model with 4 vans
**************
All Covered!
